In [1]:
import os
from os import path
import pandas as pd
import sys
from zipfile import ZipFile

In [2]:
import nibabel as nib

In [3]:
subnum = 'CC0016_core1'
typ = 'bm'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
outdir='/Users/zeynepenkavi/Downloads/ConteQC'

In [4]:
editp = path.join(basedir, 'sub-'+subnum)
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
typ_dict = {'bm': 'brainmask', 'wm': 'wm', 'bfs': 'brain.finalsurfs'}
vol = typ_dict[typ]

In [9]:
bm_edit_img = nib.load(path.join(editp,'mri/%s.mgz')%(vol))
bm_edit_data = bm_edit_img.get_fdata()

In [11]:
bm_unedit_img = nib.load(path.join(uneditp,'sub-%s/mri/%s.mgz')%(subnum, vol))
bm_unedit_data = bm_unedit_img.get_fdata()

In [12]:
diff_data = bm_unedit_data - bm_edit_data

In [14]:
import numpy as np

In [28]:
bm_edit_data.shape

(256, 256, 256)

In [42]:
diff_data[np.where(diff_data != 0)]

#Slice order is Sag, Axe, Cor

array([110.,  79.,  68.,  86.,  78.,  62.,  59.,  96., 115., 118.,  85.,
        86.,  55.,  72., 127.,  95.,  84.,  50.,  88., 101., 125.,  63.,
        74.,  67.,  92., 122.,  75.,  87.,  61.,  51.,  54.,  61.,  67.,
        92., 105.,  65.,  62.,  58.,  74.,  76.,  67.,  87.,  57.,  63.,
        66.,  78.,  83.,  70.,  60.,  62.,  77.,  72.,  64.,  74.,  75.,
        68.,  69.,  60.,  64.,  70.,  77.,  76.,  65.,  74.,  81.,  82.,
        76.,  69.,  78., 100.,  92.,  84.,  75.,  74., 101.,  91.,  84.,
        77.,  67.,  92.,  81., 102.,  78.,  99., 100.,  69.,  68.,  66.,
       103.,  66.,  64.,  64.,  77.,  91.,  87.,  63.,  58.,  78.,  73.,
        70.,  71.,  69.,  79.,  72.,  68.,  70.,  73.,  74.,  67.,  74.,
        78.,  76.,  61.,  71.,  76.,  73.,  75.,  78.,  77.,  73.,  62.,
        75.,  78.,  74.,  59.,  68.,  57.,  70.,  62.,  65.,  67.,  68.,
        76.,  72.,  68.,  78.,  79.,  65.,  55.,  71.,  77.,  62.,  64.,
        74.,  75.,  69.,  72.,  65.,  68.,  75.,  7

In [79]:
out = pd.DataFrame(np.asarray(np.asarray(diff_data != 0).nonzero()).T).rename(columns={0:"Sag", 1:"Axe", 2:"Cor"})

In [80]:
out['diff_val'] = diff_data[np.where(diff_data != 0)]

In [81]:
out['Action'] = np.where(out.diff_val>0, "delete voxel", "add voxel")

In [82]:
out['Vol'] = vol

In [83]:
out = out.drop(columns="diff_val")

In [84]:
out = out.sort_values(by=['Cor'])

In [90]:
out = out.reset_index(drop=True)

In [91]:
out

,Sag,Axe,Cor,Action,Vol
0,106,103,40,delete voxel,brainmask
1,112,102,40,delete voxel,brainmask
2,112,101,40,delete voxel,brainmask
3,111,102,40,delete voxel,brainmask
4,111,101,40,delete voxel,brainmask
...,...,...,...,...,...
700,174,110,161,delete voxel,brainmask
701,168,113,161,delete voxel,brainmask
702,172,113,161,delete voxel,brainmask
703,169,113,161,delete voxel,brainmask


In [92]:
#Make distance matrix
from scipy.spatial import distance_matrix, distance

#Apply clustering on the distance matrix

In [98]:
#Split by action before calculating distances
#This should only apply to wm where wm voxels might both be added or removed
dm = distance_matrix(out[['Sag', 'Axe', 'Cor']], out[['Sag', 'Axe', 'Cor']])

In [105]:
from sklearn.cluster import DBSCAN

In [109]:
clustering = DBSCAN(eps=50, min_samples=5).fit(dm)

In [138]:
out['cluster'] = clustering.labels_
out

,Sag,Axe,Cor,Action,Vol,cluster
0,106,103,40,delete voxel,brainmask,0
1,112,102,40,delete voxel,brainmask,0
2,112,101,40,delete voxel,brainmask,0
3,111,102,40,delete voxel,brainmask,0
4,111,101,40,delete voxel,brainmask,0
...,...,...,...,...,...,...
700,174,110,161,delete voxel,brainmask,12
701,168,113,161,delete voxel,brainmask,12
702,172,113,161,delete voxel,brainmask,12
703,169,113,161,delete voxel,brainmask,12


In [135]:
out_grouped = out.groupby('cluster')

out_grouped = out_grouped.agg(min_sag=('Sag', min),
               max_sag=('Sag', max),
               min_axe=('Axe', min),
               max_axe=('Axe', max),
               min_cor=('Cor', min),
               max_cor=('Cor', max),
               num_vox=('Cor', 'count')).reset_index()

In [137]:
out_grouped['vol'] = out.Vol.unique()[0]
out_grouped['action'] = out.Action.unique()[0]
out_grouped

,cluster,min_sag,max_sag,min_axe,max_axe,min_cor,max_cor,num_vox,vol,action
0,0,106,112,101,106,40,41,26,brainmask,delete voxel
1,1,138,142,126,131,45,47,22,brainmask,delete voxel
2,2,122,129,120,126,46,47,30,brainmask,delete voxel
3,3,168,176,124,130,52,56,52,brainmask,delete voxel
4,4,132,136,112,118,63,68,30,brainmask,delete voxel
5,5,130,132,100,106,73,82,60,brainmask,delete voxel
6,6,123,128,37,41,83,85,21,brainmask,delete voxel
7,7,142,146,119,123,88,96,55,brainmask,delete voxel
8,8,107,113,122,126,97,100,36,brainmask,delete voxel
9,9,145,151,124,132,98,104,62,brainmask,delete voxel
